# Online monitor example

Jelle, May 2018

This notebooks shows a basic (cS1, cS2) plot that updates while the DAQ data is incoming. To run it, first start
  * `python fake_daq.py`
  * `python eb.py --mongo`
(each in a separate terminal).

At the moment you can only start the plot *after* eb.py has put in a chunk of events; you'll get an error otherwise. 

In [ ]:
import time
import itertools

import matplotlib.pyplot as plt
import strax

In [ ]:
st = strax.Context(register_all=strax.xenon.plugins,
                   storage=strax.MongoStore('mongodb://localhost'))

run_id = '180423_1021'

In [ ]:
%matplotlib notebook

max_delay = 30 * 1e9

for i in itertools.count():
    # Get all events from mongo
    df = st.get_df(run_id, ['event_basics', 'events'])
    
    # Select recent events
    now = time.time()
    df['delay'] = int(now) * int(1e9) - df['time']
    df = df[df['delay'] < int(max_delay)]

    # Update the plot
    plt.gca().clear()
    
    plt.scatter(df.s1_area, df.s2_area,
                c=df.s1_area_fraction_top,
                vmin=0, vmax=0.3,
                s=100 * (max_delay - df.delay)/max_delay,
                cmap=plt.cm.jet,
                marker='.', edgecolors='none')
    if i == 0:
        plt.colorbar(label="S1 area fraction top", extend='max')
    plt.xlabel('S1 (PE)')
    plt.ylabel('S2 (PE)')
    plt.xscale('symlog')
    plt.yscale('log')
    plt.ylim(1e2, 1e7)
    plt.xlim(-0.1, 1e6)
    plt.title(time.strftime('%H:%M:%S'))
    
    plt.gcf().canvas.draw()
    
    # Sleep 4x as long as it took to make the plot
    time.sleep(4 * (time.time() - now))
    